In [ ]:
from huggingface_hub import hf_hub_download, snapshot_download
repo_path = snapshot_download(repo_id="meta-llama/Llama-3.2-3B")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\khatt\Documents\AmazonEcho-NLP-Project\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khatt\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_H

In [1]:
print("HelloWorld")

HelloWorld


In [122]:
import os
from dotenv import load_dotenv
load_dotenv()
model_name = "E:\\models\\pleasRAG"
token = os.getenv("HFTOKEN")
import torch

In [123]:
from transformers import AutoModelForCausalLM 
from transformers import AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [140]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    return_full_text=True
)

Device set to use cpu


In [125]:
pipe("The key to life is")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'The key to life is love. (Matthew 20:22)\n\nThe question of whether or not love is a good thing, a good principle, is one that needs to be wrestled with. The most that we can say about love is that it is the most important of the many basic principles. Love is the most basic of all human principles. In the words of St. Paul, love is the love of our neighbors.\n3 [I was] a man of the world and, above all, a man of action. I knew nothing of the world, and no one knew me. I became one of the world’s most remarkable men.\n[C]haracter:\n\nI became the second man of the world, I was the second man of action. I became an Italian in the world, and a man of the world in action.\n[D]eath:\n\nThe first day of the year I was murdered. I was murdered by the death of a handful of Italian fascists who wanted to stop me from becoming the second man of the world.\n[E]xile:\n\nI lived in exile for a year.\n[F]ascism:\n\nI was fascist for'}]

In [61]:
import pandas as pd
df = pd.read_csv("Amazon Echo 2 Reviews.csv")

In [62]:
df = df.head(30)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "E:/models/gte-small"


In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    
)


C:\Users\khatt\AppData\Local\Temp\ipykernel_22340\2104233462.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE = os.getenv("PINECONE")
print(PINECONE)

pcsk_5v561g_8Y3ZthdWE8zpveZ9s6Mm4RUVH7Rnyxm6NxRJgDVdnJKGdf5khDtYhxhhA49rPdR


In [ ]:
vectorized_input = embedding_model.embed_query("This is a test")

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE)
index = pc.Index("Echo")

In [63]:
data = []
for i in range(len(df)):
    text = df['Review Text'][i]
    vector = embedding_model.embed_query(text)
    data.append(
        {
        "id":"vec{}".format(i),
        "values":vector,
        "metadata":{"text":text}
    })

In [65]:
index.upsert(
    vectors=data,
    namespace= "echo"
)

KeyboardInterrupt: 

In [133]:
prompt = """
<|system|>
You are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
{context}
---Here is the question you need to answer.

Question: {question}
<|assistant|>


"""

In [ ]:
user_input = input("User: ")

vectorized_input = embedding_model.embed_query(user_input)

context = index.query(
    namespace="echo",
    vector=vectorized_input,
    top_k=2,
    include_metadata=True
)


In [134]:
combinedcontext = [context['matches'][i]['metadata']['text'] for i in range(len(context['matches']))]
combinedcontext=str(".".join(combinedcontext))
combinedcontext

'Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.'

In [141]:
answer = pipe(prompt.format(context = combinedcontext, question = user_input))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [142]:
answer

[{'generated_text': '\n<|system|>\nYou are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.\nGive the rational and well written response. If you don\'t have proper info in the context, answer "I don\'t know"\nRespond only to the question asked.\n\n<|user|>\nContext:\nAwesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.\n---Here is the question you need to answer.\n\nQuestion: do people like the product?\n<|assistant|>\n\n\nRescued by the context, I can provide a response based on the available information. The product is indeed a great one. Alexa will do her bes

In [143]:
print(answer[0]['generated_text'])


<|system|>
You are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.
---Here is the question you need to answer.

Question: do people like the product?
<|assistant|>


Rescued by the context, I can provide a response based on the available information. The product is indeed a great one. Alexa will do her best to process this query as a Spanish l